In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 5. 학습 알고리즘 구현하기
    1. 미니배치 
        => 훈련 데이터 중 일부를 무작위로 가져온다 
        => 이렇게 선별한 데이터=> 미니배치
        => 그 미니배치의 손실 함수 값을 줄이는게 목표 
    2. 기울기 산출 
        => 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 
        매개변수의 기울기를 구합니다. 
        => 기울기는 손실 함수의 값을 가장 작게하는 방향을 제시 
        
    3. 매개변수 갱신 
        => 가중치 매개변수를 기울기 방향으로 아주 조금 갱신합니다
        
    4. 반복 
        => 1~3단계 반복

### 위의 순서는 확률적 경사하강법(SGD)
    => 확률적으로 무작위로 골라낸 데이터

In [15]:
# 교차 엔트로피 오차 
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [16]:
def softmax(x):
    array_x = x - np.max(x)
    
    exp_x = np.exp(array_x)
    result = exp_x / np.sum(exp_x)
    
    return result

In [17]:
def sigmoid(x):
    return 1 / 1+ np.exp(-x)

In [18]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성 
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h) 계산 
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val #값 복원 
        
    return grad
        
        

### 2층 신경망 클래스 구현하기

In [55]:
# 하나의 class 로 구현
class TwoLayerNet:
    
    # 초기화 수행 메서드
    # params 는 신경망의 매개변수를 보관하는 딕셔너리 변수(인스턴스 변수)
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {} #가중치 초기화
        # (입력 * 은닉층) 행렬의 가중치 생성
        self.params['w1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        
        # 은닉층 노드수만큼 편향 생성, 0으로 설정 
        self.params['b1'] = np.zeros(hidden_size)
        
        self.params['w2'] = weight_init_std * np.random.randn(hidden_size, input_size)
        
        self.params['b2'] = np.zeros(output_size)
        
        
    # 예측(추론) 수행 메섣, x: 이미지 데이터
    def predict(self, x):
        w1, w2 = self.params['w1'], self.params['w2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, w1) + b1
        z1 = sigmoid(a1)
        
        a2 = np.dot(z1, w2) + b2
        y = softmax(a2)
        
        return y
    
    # 손실함수 값을 구하는 메서드 x:입력 데이터, t: 정답 레이블 
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    # 정확도 측정 메서드 
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1) # y행렬의 1차원 요소 최대값 추출
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # 가중치 매개변수의 기울기를 구하는 메서드 
    #grads 는 기울기를 보관하는 딕셔너리 변수 (numerical_gradient의 반환값)
    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)
        
        grads = {}
        grads['w1'] = numerical_gradient(loss_w, self.params['w1'])
        grads['b1'] = numerical_gradient(loss_w, self.params['b1'])
        
        grads['w2'] = numerical_gradient(loss_w, self.params['w2'])
        grads['b2'] = numerical_gradient(loss_w, self.params['b2'])
        
        return grads

In [56]:
from dataset.mnist import load_mnist

## 미니배치 학습 구현하기

In [57]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, 
                                                 one_hot_label=True)
train_loss_list=[]

In [58]:
x_train.shape, t_train.shape

((60000, 784), (60000, 10))

In [59]:
x_test.shape, t_test.shape

((10000, 784), (10000, 10))

In [60]:
# 하이퍼파라미터 (사용자가 지정해줘야하는 변수)
iters_num =10000 # 반복횟수 
train_size = x_train.shape[0] # 데이터 개수 shape[0] , 행의개수
batch_size = 100 #미니배치 크기 
learning_rate = 0.1 #학습률
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10) # 은닉층 적절히 임의로


In [61]:
for i in range(iters_num):
    # 미니배치 획득 
    
    #훈련데이터 개수에서, batch_size만큼 랜덤선택
    #인덱스만 선택이 된다
    batch_mask = np.random.choice(train_size, batch_size) 
    
    #그 인덱스들을 넣어서 임의의 x, t 데이터 선택 
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산 
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # 매개변수 갱신 
    for key in ('w1','b1','w2','b2'):
        network.params[key] -= learning_rate * grad[key]
        
    # 학습 경과 기록 
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    

ValueError: operands could not be broadcast together with shapes (100,784) (10,) 